In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import window,column,desc,col,instr,expr, pow,translate,lit
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import unix_timestamp, from_unixtime
import pandas as pd

spark = SparkSession \
    .builder \
    .appName("Foo") \
    .config("spark.executor.memory", "1g") \
    .config("spark.driver.memory", "1g") \
    .getOrCreate()
import warnings
warnings.filterwarnings('ignore')

In [3]:
# print all the outputs in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Penalties: You will incur penalties if:

* Your code is wrong
* Your code would not work on different data ( don't use any hard-coded code)
* Your code is unnecessarily slow (you use a for loop, or you use DataFrame.apply unnecessarily, etc)
* You answer is composed of more than one output, unless explicitly permitted

### Data Set:  San Francisco Crime Data

In [4]:
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").option("delimiter", ",")\
.option("dateFormat", "MM/dd/YYYY HH:mm")\
.load("C:/Users/ramya/Desktop/Santa Clara University/Q3/Pyspark/SF_crime_1.csv")

df.show(3)

+-------------------+--------------+--------------------+---------+----------+--------------+--------------------+
|              Dates|      Category|            Descript|DayOfWeek|PdDistrict|    Resolution|             Address|
+-------------------+--------------+--------------------+---------+----------+--------------+--------------------+
|2015-05-13 23:53:00|      WARRANTS|      WARRANT ARREST|Wednesday|  NORTHERN|ARREST, BOOKED|  OAK ST / LAGUNA ST|
|2015-05-13 23:53:00|OTHER OFFENSES|TRAFFIC VIOLATION...|Wednesday|  NORTHERN|ARREST, BOOKED|  OAK ST / LAGUNA ST|
|2015-05-13 23:33:00|OTHER OFFENSES|TRAFFIC VIOLATION...|Wednesday|  NORTHERN|ARREST, BOOKED|VANNESS AV / GREE...|
+-------------------+--------------+--------------------+---------+----------+--------------+--------------------+
only showing top 3 rows



In [4]:
df.count()

878049

## Q1

### Q1.1, How many crime instances this data set covers? (2.5 pts)

In [5]:
df.count()

878049

### Q1.2, Is there any Missing data /NaN exists? (2.5 pts)

In [6]:
df.select([sum(when(col(c).isNull(),1).otherwise(0)).alias(c) for c in df.columns]).show()

+-----+--------+--------+---------+----------+----------+-------+
|Dates|Category|Descript|DayOfWeek|PdDistrict|Resolution|Address|
+-----+--------+--------+---------+----------+----------+-------+
|    0|       0|       0|        0|         0|         0|      0|
+-----+--------+--------+---------+----------+----------+-------+



## Q2

### Q2.1, In this data set, how many different crime categories are there?  (2.5 pts)

In [7]:
df.select(col("Category")).distinct().count()

39

### Q2.2, Follow last question, list the most frequent happen top 5 ones with the crime name and their instances count.   (2.5 pts)

In [10]:
df.groupBy("Category").count().orderBy(desc("count")).show(5)

+--------------+------+
|      Category| count|
+--------------+------+
| LARCENY/THEFT|174900|
|OTHER OFFENSES|126182|
|  NON-CRIMINAL| 92304|
|       ASSAULT| 76876|
| DRUG/NARCOTIC| 53971|
+--------------+------+
only showing top 5 rows



## Q3

### Q3.1, What's the first(earliest) date this data set starts? (1.5 pts)

In [11]:
df.printSchema()

root
 |-- Dates: timestamp (nullable = true)
 |-- Category: string (nullable = true)
 |-- Descript: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- PdDistrict: string (nullable = true)
 |-- Resolution: string (nullable = true)
 |-- Address: string (nullable = true)



In [30]:
df.agg(min("Dates")).show()

+-------------------+
|         min(Dates)|
+-------------------+
|2003-01-06 00:01:00|
+-------------------+



### Q3.2, What's the last date? (1.5 pts)

In [42]:
df.agg(max("Dates")).collect()[0][0]-df.agg(min("Dates")).collect()[0][0]

datetime.timedelta(days=4510, seconds=85920)

### Q3.3, Total it covers how many days? (2 pts)

Or..

In [19]:
(df.index[0] - df.index[-1]).days

4510

## Q4

### Q4.1, If converts PdDistrict into dummy columns. after the conversion, how many columns this new data frame will have ? (2.5 pts)
(Note: Use a new name for this newly generated dataframe)

In [35]:
df.select([countDistinct(c).alias(c) for c in df.columns]).show()

+------+--------+--------+---------+----------+----------+-------+
| Dates|Category|Descript|DayOfWeek|PdDistrict|Resolution|Address|
+------+--------+--------+---------+----------+----------+-------+
|389241|      39|     879|        7|        10|        17|  23228|
+------+--------+--------+---------+----------+----------+-------+



### Q4.2, After PdDistrict got converted into dummy columns, which other column is the best candidate for further dummy conversion and why ?  (2.5 pts)

The 'DaysOfWeek' will be the best candidate because it only introduce 7 new dummy columns. The df is still remain 'human readiable'.

## Q5

### Q5.1, Which day of week has the most crime instances? (2.5 pts)

In [37]:
df.groupBy("DayOfWeek").count().orderBy(desc("count")).first()

Row(DayOfWeek='Friday', count=133734)

### Q5.2, Based on last question( in that day of week), list the crime categories that has more than 10,000 instances. (list both the category and the count)  (2.5 pts)
Note: do not use hardcoded day of week

In [46]:
df.filter(col("DayOfWeek")=="Friday").groupBy("Category").count().filter(col("count")>=10000).orderBy(desc("count")).show()

+--------------+-----+
|      Category|count|
+--------------+-----+
| LARCENY/THEFT|27104|
|OTHER OFFENSES|18588|
|  NON-CRIMINAL|13984|
|       ASSAULT|11160|
+--------------+-----+



## Q6

### Q6.1, How many crime instances happened during year 2008? (2.5 pts)

In [50]:
df.filter(year(col("Dates"))=="2008" ).count()

70174

### Q6.2, Based on last question, how many of them are happened on Saturday? (2.5 pts)

In [53]:
df.filter(year(col("Dates"))=="2008").filter(col("DayOfWeek")=="Saturday").count()

10648

## Q7

### Q7.1, Which district has the highest DRUG/NARCOTIC problem? (5 pts)

In [55]:
df.filter(df.Category=='DRUG/NARCOTIC').groupBy("PdDistrict").count().orderBy(desc("count")).show(1)

+----------+-----+
|PdDistrict|count|
+----------+-----+
|TENDERLOIN|17696|
+----------+-----+
only showing top 1 row



### Q7.2, Based on last question, what's the ratio for DRUG/NARCOTIC instances compare to all crime instances in that district? (5 pts)

In [56]:
df.filter(col("PdDistrict")=="TENDERLOIN").count()

81809

In [58]:
df.filter(col("PdDistrict")=="TENDERLOIN").filter(col("Category")=='DRUG/NARCOTIC').count()/df.filter(col("PdDistrict")=="TENDERLOIN").count()

0.2163087190895867

## Q8

### Q8.1, Let's look into different Districts. What's the district with the highest crime count ?  (3 pts)

In [59]:
df.groupBy("PdDistrict").count().orderBy(desc("count")).show(1)

+----------+------+
|PdDistrict| count|
+----------+------+
|  SOUTHERN|157182|
+----------+------+
only showing top 1 row



### Q8.2, Total how many districts have the unresolved rate higher than the average unresolved rate? (3 pts)
(hint: unresolved equal to NONE in Resolution)

In [62]:
avg_rate=df.filter(df.Resolution=='NONE').count()/df.count()

In [65]:
df.groupBy("PdDistrict").agg(mean(when(col("Resolution")=="NONE",1).\
         otherwise(0)).alias("c")).sort("c",ascending=False).filter(col("c")>=avg_rate).count()

6

### Q8.3, Based on last question, what are those districts?  (4 pts)

In [67]:
df.groupBy("PdDistrict").agg(mean(when(col("Resolution")=="NONE",1).\
         otherwise(0)).alias("c")).sort("c",ascending=False).filter(col("c")>=avg_rate).select("PdDistrict").show()

+----------+
|PdDistrict|
+----------+
|  RICHMOND|
|   CENTRAL|
|   TARAVAL|
|  NORTHERN|
| INGLESIDE|
|      PARK|
+----------+



## Q9

### Q9.1, How many crime instances are related to GUN ?  (5 pts)
(hint: looks into Description)

In [68]:
df.filter(instr(upper(col("Descript")),"GUN")>=1).count()

7727

### Q9.2, Based on last question, list the top 2 categories and their number of instances. (5 pts)

In [69]:
df.filter(instr(upper(col("Descript")),"GUN")>=1).groupBy("Category").count().orderBy(desc("count")).show(2)

+--------+-----+
|Category|count|
+--------+-----+
| ROBBERY| 4906|
| ASSAULT| 1618|
+--------+-----+
only showing top 2 rows



## Q10

### Q10.1, Let's look into SUICIDE cases. What's the day of week suicide most frequently happen? (3 pts)

In [70]:
df.filter(col("Category")=="SUICIDE").groupBy("DayOfWeek").count().orderBy(desc("count")).first()

Row(DayOfWeek='Thursday', count=89)

### Q10.2, Based on last question(consider only that day of week), what' the district happen the most ? (3 pts)

In [72]:
df.filter(col("DayOfWeek")=="Thursday").filter(col("Category")=="SUICIDE").\
groupBy("PdDistrict").count().orderBy(desc("count")).first()

Row(PdDistrict='MISSION', count=14)

### Q10.3, Base on last 2 questions, what' the porpotion by JUMPING ? (4 pts)

In [78]:
df.filter(col("DayOfWeek")=="Thursday").filter(col("Category")=="SUICIDE").filter(col("PdDistrict")=="MISSION").\
agg(mean(when(instr(lower(df.Descript),"jumping")>=1,1).\
         otherwise(0)).alias("c")).show()

+-------------------+
|                  c|
+-------------------+
|0.14285714285714285|
+-------------------+



## Q11

### Q11.1, Let's investigate the 'TRAFFIC VIOLATION'. What's the total number of violation instance?  (3 pts)

In [81]:
df.filter(instr(upper(col("Descript")),"TRAFFIC VIOLATION")>=1).count()

21522

### Q11.2,  For all traffic violation instances, find the top address where traffic violation happen the most.  (3 pts)

In [83]:
df.filter(instr(upper(col("Descript")),"TRAFFIC VIOLATION")>=1).groupBy("Address").count().orderBy(desc("count")).first()

Row(Address='800 Block of BRYANT ST', count=372)

### Q11.3, For all traffice violation instances, how many instances happen on Address with 'GOLDEN GATE AV' as the leading cross street? (4 pts)

In [84]:
df.filter(col("Address").startswith("GOLDEN GATE AV")).filter(instr(upper(col("Descript")),"TRAFFIC VIOLATION")>=1).count()

121

Note: If the question asks traffic violation address associate with all instnces with 'GOLDEN GATE AV', then should use .apply(lambda y: 'GOLDEN GATE AV' in y)

## Q12

### Q12.1, For each police district, find the top 1 crime category and the number of instances it happened.  Sort the table based on the number of instances descending. (10 pts)
(note: Your output need PdDistrict, Category and the number of counts all 3 listed in one table.)

If use only one district, say BAYVIEW, write the code

In [21]:
from pyspark.sql.functions import row_number
from pyspark.sql import Window
import pyspark.sql.functions as f

In [11]:
grouped_data=df.groupBy("PdDistrict","Category").count().orderBy(desc("count"))

In [14]:
grouped_data.show(3)

+----------+-------------+-----+
|PdDistrict|     Category|count|
+----------+-------------+-----+
|  SOUTHERN|LARCENY/THEFT|41845|
|  NORTHERN|LARCENY/THEFT|28630|
|   CENTRAL|LARCENY/THEFT|25060|
+----------+-------------+-----+
only showing top 3 rows



In [26]:
w = Window.partitionBy('PdDistrict')
grouped_data.withColumn("max_d",f.max('count').over(w)).where(f.col('count') == f.col('max_d'))\
    .drop('max_d').orderBy(desc("count")).show()

+----------+--------------+-----+
|PdDistrict|      Category|count|
+----------+--------------+-----+
|  SOUTHERN| LARCENY/THEFT|41845|
|  NORTHERN| LARCENY/THEFT|28630|
|   CENTRAL| LARCENY/THEFT|25060|
|   MISSION|OTHER OFFENSES|19330|
|TENDERLOIN| DRUG/NARCOTIC|17696|
|   BAYVIEW|OTHER OFFENSES|17053|
| INGLESIDE|OTHER OFFENSES|13203|
|   TARAVAL| LARCENY/THEFT|11845|
|  RICHMOND| LARCENY/THEFT| 9893|
|      PARK| LARCENY/THEFT| 9146|
+----------+--------------+-----+



## Q13

### Q13.1, Now let's define the LARCENY/THEFT, ASSAULT, WARRANTS and ROBBERY all these 4 categories count as 'major crime'. For each police district, find the total major crime count and the  porpotion of these major crime got resolved as  'ARREST, BOOK' and 'ARREST, CITED' from the Resolution. Rename those two columns as 'Major Crime Count' and 'Arrest Rate', then sort the output data frame based on 'Arrest Rate' decending.  (10 pts)

Create new column based on those 4 categories

In [29]:
df=df.withColumn("major crime", when((col("Category")=='LARCENY/THEFT') | \
                                     (col("Category")=='ASSAULT') | \
                                     (col("Category")=='WARRANTS') |\
                                     (col("Category")=='ROBBERY')\
                                     , 1.0).otherwise(0.0))

In [38]:
df=df.withColumn("Arrest Rate",when((col("Resolution")=="ARREST, BOOKED") | (col("Resolution")=="ARREST, CITED"),1).otherwise(0))

In [39]:
df.printSchema()

root
 |-- Dates: timestamp (nullable = true)
 |-- Category: string (nullable = true)
 |-- Descript: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- PdDistrict: string (nullable = true)
 |-- Resolution: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- major crime: double (nullable = false)
 |-- Arrest Rate: integer (nullable = false)



In [46]:
df.filter(col("major crime")==1).groupBy("PdDistrict").agg(sum("major crime").alias("Major Crime Count"),\
                             round(mean("Arrest Rate"),3).alias("Arrest Rate")).orderBy(desc("Arrest Rate")).show()

+----------+-----------------+-----------+
|PdDistrict|Major Crime Count|Arrest Rate|
+----------+-----------------+-----------+
|TENDERLOIN|          27110.0|      0.501|
|   MISSION|          39616.0|      0.323|
|   BAYVIEW|          27013.0|      0.294|
|  SOUTHERN|          67008.0|      0.266|
| INGLESIDE|          24085.0|      0.258|
|      PARK|          15936.0|      0.254|
|   TARAVAL|          20330.0|      0.237|
|  NORTHERN|          44187.0|      0.196|
|   CENTRAL|          36813.0|      0.188|
|  RICHMOND|          14892.0|      0.168|
+----------+-----------------+-----------+

